In [1]:
import pandas as pd
import numpy as np
import requests
from time import sleep
import itertools
import datetime
from tqdm import tqdm
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def get_airports (Access_Token)  :
    
    #
    authentification = {}
    authentification["Authorization"] = "Bearer " + Access_Token
    
    #
    df_list = []
    offset_list = list(range(0,1500,100))
    offset_list[offset_list.index(100)]= 170
    offset_list[offset_list.index(1000)]= 1005
    
    for i in offset_list:
        sleep(0.1)
        reponse = requests.get(("https://api.lufthansa.com/v1/mds-references/airports/?lang=AN&limit=100&offset={offset}&LHoperated=1").format(offset = i), headers = authentification).json()
        df_list.append(pd.DataFrame(reponse['AirportResource']['Airports']['Airport']))

    df = pd.concat(df_list)  
    df = df.reset_index(drop=True) 

    for i in range(len(df)):
        airport_name = df.loc[i, "Names"].get("Name", {}).get("$",)
        df.loc[i, "Names"] = airport_name


        latitude = df.loc[i, "Position"]["Coordinate"].get("Latitude")
        df.loc[i, "Latitude"] = latitude

        longitude = df.loc[i, "Position"]["Coordinate"].get("Longitude")
        df.loc[i, "Longitude"] = longitude



    Airports = df.drop("Position", axis= 1).drop_duplicates().reset_index(drop=True)
    
    return Airports
    

In [4]:
airports = get_airports ("8f6hdypb5qtaqfdarwuc27hr")

In [5]:
#airports.head()
european_codes = ["AL", "AD", "AT", "BY", "BE", "BA", "BG", "HR", "CY", "CZ", "DK", "EE", "FI", "FR", "DE", 
                  "GR", "HU", "IS", "IE", "IT", "XK", "LV", "LI", "LT", "LU", "MK", "MT", "MD", "MC", "ME", 
                  "NL", "NO", "PL", "PT", "RO", "RU", "SM", "RS", "SK", "SI", "ES", "SE", "CH", "UA", "GB"]
europe = airports[airports['CountryCode'].isin(european_codes)].reset_index(drop=True)["AirportCode"]

In [6]:
origins = europe
destinations = europe

start_date = ["2023-04-26"]
num_days = 1



In [7]:
import numpy as np

combinations = list(itertools.product(origins, destinations, start_date))

np.random.seed(123)

ousmane , abdoul , abdel = np.array_split(np.random.permutation(combinations), 3)


In [8]:
from tqdm import tqdm

def get_flight_schedules(Access_Token, combinaisons):
    
    authentification = {}
    authentification["Authorization"] = "Bearer " + Access_Token

    df_list = []
    counter = 0

    for combination in tqdm(combinaisons):  
        try:
            response = requests.get(
                ("https://api.lufthansa.com/v1/operations/schedules/{origin}/{destination}/{date}"
                 "?directFlights=0&limit=100&offset=0"
                ).format(origin = combination[0], destination = combination[1], date = combination[2]),
                headers=authentification).json()

            for element in response['ScheduleResource']['Schedule']:
                for flight in element.get('Flight', []):
                    departure = flight.get('Departure', {})
                    arrival = flight.get('Arrival', {})
                    carrier = flight.get('MarketingCarrier', {})
                    equipment = flight.get('Equipment', {})
                    details = flight.get('Details', {})

                    df = pd.DataFrame({
                          'DepAirportCode': departure.get('AirportCode'),
                          'DepScheduledTimeLocal': departure.get('ScheduledTimeLocal', {}).get('DateTime'),
                          'ArrAirportCode': arrival.get('AirportCode'),
                          'ArrScheduledTimeLocal': arrival.get('ScheduledTimeLocal', {}).get('DateTime'),
                          'Terminal': arrival.get('Terminal', {}).get('Name'),
                          'AirlineID': carrier.get('AirlineID'),
                          'FlightNumber': carrier.get('FlightNumber'),
                          'AircraftCode': equipment.get('AircraftCode'),
                          'DaysOfOperation' : details.get('DaysOfOperation',),
                          'Effective' : details.get('DatePeriod',).get('Effective'),
                          'Expiration' : details.get('DatePeriod',).get('Expiration')                  
                      }, index=[0])

                    df_list.append(df[df['AirlineID'] == "LH"].reset_index(drop=True))

            counter += 1

            # Pause de 0,2 seconde toutes les 5 requetes
            if counter % 5 == 0:
                time.sleep(0.2)

        except:
            continue

    FlightSchedules = pd.concat(df_list).drop_duplicates().reset_index(drop=True)
    FlightSchedules.insert(0, "FlightScheduleID" , FlightSchedules.index+1)
    
    return FlightSchedules

In [ ]:
df = df = get_flight_schedules(Access_Token, prenom)
df.to_csv("/content/drive/MyDrive/europe_europe.csv", index=False)